# SQL Query Agent in LangChain

Langchain provides a SQL Agent which offers a flexible way of interacting with SQL Databases. The SQL Agent system can answer questions intelligently by utilizing both the schema and content of the database, allowing it to provide insights into specific tables or records as needed. It can effectively handle and recover from errors by executing a generated query, catching any errors in the traceback, and adjusting the query accordingly. The system is optimized to query the database as many times as necessary to gather the information required to answer user questions fully. Additionally, it conserves resources by only retrieving the schema of relevant tables, ensuring efficient use of tokens and improving performance.


## Lab Description:

This lab explores how to use LangChain’s SQL Query Agent to interact with a SQLite database (chinook.db). Participants will learn two methods to fetch table names and retrieve table information before using the SQL Agent to execute queries. The lab demonstrates how the agent can automatically generate SQL queries, retrieve data, and provide table descriptions, making database interactions more efficient and intuitive.

## Lab Objectives

- Learn how to connect LangChain to a SQLite database (chinook.db).
- Explore methods to fetch table names and retrieve table information.
- Utilize LangChain’s SQL Query Agent to execute queries dynamically.
- Generate and interpret table descriptions using the SQL Agent.

## Load the LLM 

First step is to load the LLM. We use gemma2 9 billion parameter model. 

In [3]:
from langchain_ollama.llms import OllamaLLM

In [45]:
llm = OllamaLLM(model = "gemma2:9b", base_url="http://10.79.253.112:11434", temperature=0)

## The Langchain SQL Database Toolkit

The `SQLDatabaseToolkit` contains tools that helps interact with a `SQL` database. `SQLDatabaseToolkit` enables SQL Agents to answer questions using data in a database. `.from_uri` method constructs a SQLAlchemy engine from URI. The URI string specifies the path to the database. The database should be located in the same directory as the notebook. We use SQLLite as it is lightweight and serverless. Working with agents are a lot easier when using lightweight SQL Engines. 

In [5]:
from langchain_community.utilities import SQLDatabase

In [11]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

## `get_usable_table_names` Method

The `get_usable_table_names` returns the names of all tables in the database.

In [9]:
db.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

## `get_table_info` Method

`get_table_info` method can be called to fetch details about specific tables. 

In [12]:
print(db.get_table_info(["Album"]))


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


This returned a description about the Albums table. `get_table_info` accepts a list of strings so that description about multiple tables can be fetched. 

In [13]:
print(db.get_table_info(["Album", "Artist"]))


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


## The `create_sql_agent` Method

`create_sql_agent` is used to create a SQLAgent from an LLM and database. We provide the LLM, and the db object we initialized to the `create_sql_agent`method. 

In [39]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

In [50]:
agent = create_sql_agent(llm, db=db)

Now that we have initialized the agent, we can invoke it with the `.invoke` method. 

In [51]:
result = agent.invoke("How many different Artists are in the database?")

In [52]:
result

{'input': 'How many different Artists are in the database?',
 'output': 'There are 275 different artists in the database.'}

The agent generated a response by querying the database. 

We can set `verbose=True` to see the step by step thought process of the agent. 

In [57]:
agent = create_sql_agent(llm, db=db, verbose=True)

In [59]:
agent.invoke("List the total sales per country")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: The 'Invoice' table likely contains sales information and might have a country associated with it. I should query the schema of the 'Invoice' table.
Action: sql_db_schema
Action Input: Invoice
CREATE TABLE "Invoice" (
	"InvoiceId" INTEGER NOT NULL, 
	"CustomerId" INTEGER NOT NULL, 
	"InvoiceDate" DATETIME NOT NULL, 
	"BillingAddress" NVARCHAR(70), 
	"BillingCity" NVARCHAR(40), 
	"BillingState" NVARCHAR(40), 
	"BillingCountry" NVARCHAR(40), 
	"BillingPostalCode" NVARCHAR(10), 
	"Total" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("InvoiceId"), 
	FOREIGN KEY("CustomerId") REFERENCES "Customer" ("CustomerId")
)

/*
3 rows from Invoice table:
InvoiceId	CustomerId	InvoiceDate	BillingAddress	BillingCity	BillingState	BillingCountry	BillingPostalCode	Total
1	2	2021-01-01 00:00:00	Theodor-Heuss-Straße 3

{'input': 'List the total sales per country',
 'output': 'The top 10 countries with the highest total sales are: USA (523.06), Canada (303.96), France (195.1), Brazil (190.1), Germany (156.48), United Kingdom (112.86), Czech Republic (90.24), Portugal (77.24), India (75.26), and Chile (46.62).'}

The agent identifies the relevant table based on the user query. It then analyses the schema of the identified table to generate a syntactically correct query to generate the final output. 

## Table Descriptions

SQLAgents can also describe the content of a specific table. That is, it can provide all information on what the table is about. 

In [60]:
agent.invoke("What is the 'Employee' table about ?")



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: The 'Employee' table seems like a good place to start. I should query its schema.
Action: sql_db_schema
Action Input: Employee
CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from

{'input': "What is the 'Employee' table about ?",
 'output': "The 'Employee' table stores information about employees in a company, including their ID, name, title, manager, hire date, contact details, and address."}

The agent provided a description about the Employee table. 

In [ ]:
agent.invoke("Generate query for fetching the names of all the employees")

<div style="text-align: left;">
    <img src="logo.png" alt="flow" width="150" height="100">
</div>